# Example how to use a NeXus/HDF5/NXapm file

Make sure that you have successfully created a NeXus/HDF5 file before starting this tutorial.<br>
If this was successful, define the path and name of the NeXus file you want to work with.<br>

In [ ]:
nxs_file_path = "apm.case1.nxs"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from jupyterlab_h5web import H5Web
plt.rcParams["figure.figsize"] = [20, 10]
plt.rcParams["figure.dpi"] = 300
import h5py

In [ ]:
H5Web(nxs_file_path)

Read mass-to-charge-state ratio values, create a histogram ("mass spectrum"), and mark ranges.

In [ ]:
# load data and ranges e.g. for case1
with h5py.File(nxs_file_path, "r") as h5r:
    mq = h5r["entry1/atom_probe/mass_to_charge_conversion/mass_to_charge"][:]
    nions = np.uint32(h5r["entry1/atom_probe/ranging/number_of_ion_types"][()])
    print(f"Array with mass-to-charge-state ratios loaded, {nions} iontypes are distinguished")

In [ ]:
# define binning
[mqmin, mqmax] = [0., 100.0]  # Da np.max(mq)]
print(f"Dataset ranging from [ {mqmin}, {mqmax}] Da.")
mqincr = 0.01  # Da
print(f"Using a mass-to-charge-state ratio resolution of {mqincr} Da.")

In [ ]:
# transform collection of mass-to-charge-state ratios into a histogram
hst1d = np.unique(np.uint64(np.floor((mq[np.logical_and(mq >= mqmin, mq <= mqmax)] - mqmin) / mqincr)), return_counts=True)
nbins = np.uint64((mqmax - mqmin) / mqincr + 1)
print(f"Histogram has {nbins} bins.")

In [ ]:
# use matplotlib and numpy to plot histogram data 
xy = np.zeros([nbins, 2], np.float64)
xy[:,0] = np.linspace(mqmin + mqincr, mqmax + mqincr, nbins, endpoint=True)
xy[:,1] = 0.5  # * np.ones([nbins], np.float64)  # 0.5 to be able to plot logarithm you can not measure half an atom
for i in np.arange(0, len(hst1d[0])):
    binidx = hst1d[0][i]
    xy[binidx, 1] = hst1d[1][i]
print("Mass-to-charge-state histogram created.")

In [ ]:
[xmi, xmx, ymi, ymx] = [mqmin, 10**np.ceil(np.log10(mqmax)), 0.5, 10**np.ceil(np.log10(np.max(xy[:,1])))]
[xmi, xmx, ymi, ymx] = [mqmin, mqmax, 0.5, 10**np.ceil(np.log10(np.max(xy[:,1])))]

with h5py.File(nxs_file_path, "r") as h5r:
    fig, cnts_over_mq = plt.subplots(1, 1)
    plt.plot(xy[:, 0], xy[:, 1], color="blue", alpha=0.5, linewidth=1.0)
    for i in np.arange(1, nions):
        print(f"Collect ion{i}...")
        # load ranges and plot them
        ranges = h5r[f"entry1/atom_probe/ranging/peak_identification/ion{i}/mass_to_charge_range"][:]
        for min_max in ranges:
            cnts_over_mq.vlines(min_max[0], 0, 1, transform=cnts_over_mq.get_xaxis_transform(), alpha=0.1, color="grey", linestyles="dotted")
            cnts_over_mq.vlines(min_max[1], 0, 1, transform=cnts_over_mq.get_xaxis_transform(), alpha=0.1, color="grey", linestyles="dotted")
            # rng = Rectangle((min_max[0], ymi), min_max[1] - min_max[0], ymx - ymi, edgecolor="r", facecolor="none")
    # plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9], ["Min", "0.0025", "0.025", "0.25", "0.50", "0.75", "0.975", "0.9975", "Max"])
    plt.yscale("log")
    plt.legend([r"Mass-to-charge-state ratio $\Delta\frac{m}{q} = $"+str(mqincr)+" Da"], loc="upper right")
    plt.xlabel(r"Mass-to-charge-state-ratio (Da)")
    plt.ylabel(r"Counts")
    print("Mass-to-charge-state histogram visualized.")
    # scale bar with add margin to the bottom and top of the yaxis to avoid that lines fall on x axis
    margin=0.01  # polishing the margins
    plt.xlim([-margin * (xmx - xmi) + xmi, +margin * (xmx - xmi) + xmx])
    plt.ylim([ymi, +margin * (ymx - ymi) + ymx])

In [ ]:
# save the figure
figfn = f"{nxs_file_path}.MassToChargeStateRatio.png"
fig.savefig(figfn, dpi=300, facecolor="w", edgecolor="w", orientation="landscape", format="png", 
            transparent=False, bbox_inches="tight", pad_inches=0.1, metadata=None)
# plt.close("all")
print(f"{figfn} stored to disk.")

# Conclusions:
***

This tutorial showed how you can load and work with data within an NeXus/HDF5 file that<br>
is formatted according to the NXapm application definition.

### Contact person for pynxtools-apm and related examples in FAIRmat:
Dr.-Ing. Markus Kühbach, 2024/09/12<br>

### Funding
<a href="https://www.fairmat-nfdi.eu/fairmat">FAIRmat</a> is a consortium on research data management which is part of the German NFDI.<br>
The project is funded by the Deutsche Forschungsgemeinschaft (DFG, German Research Foundation) – project 460197019.